In [8]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import random

# Load data
interactions_df = pd.read_excel('Combined_Interactions.xlsx')
events_df = pd.read_excel('Events.xlsx')
responses = pd.read_csv('(Responses) (1).csv')

# Create user-item matrix
user_item_matrix = interactions_df.pivot_table(index='Name', columns='Event ID', values='Interaction Score', fill_value=0)

# Compute user similarity
user_similarity = cosine_similarity(user_item_matrix)

def recommend_events(user, events_df, responses, add_interests=[], remove_interests=[], ratio_added_interests=0.5):
    ##### Collaborative Based Filtering

    user_index = user_item_matrix.index.get_loc(user)
    similar_users = np.argsort(user_similarity[user_index])[::-1][1:]  # Exclude the user itself

    recommended_events = pd.DataFrame(columns=['Event ID', 'score'])

    interacted_events = interactions_df[interactions_df['Name'] == user]

    for similar_user in similar_users:
        for event_id in user_item_matrix.columns:
            if user_item_matrix.loc[user_item_matrix.index[similar_user], event_id] > 0 and event_id not in interacted_events['Event ID'].tolist():
                score = user_item_matrix.loc[user_item_matrix.index[similar_user], event_id]
                if event_id not in recommended_events['Event ID'].tolist():
                    recommended_events = pd.concat([recommended_events, pd.DataFrame({'Event ID': [event_id], 'score': [score]})], ignore_index=True)
                else:
                    recommended_events.loc[recommended_events['Event ID'] == event_id, 'score'] += score

    recommended_events = recommended_events.merge(events_df, on='Event ID')
    recommended_events = recommended_events.sort_values(by='score', ascending=False)
    recommended_events.drop('score', axis=1, inplace=True)

    ##### Content Based Filtering

    responses = responses[responses['Name'] == user]
    responses['All Interests'] = responses['All Interests'].str.split(', ')
    events_df['Category'] = events_df[['Primary Category', 'Secondary Category']].apply(lambda x: ', '.join(filter(lambda y: pd.notna(y), x)), axis=1)
    events_df['Category'] = events_df['Category'].str.split(', ')

    # Incorporate additional interests
    if add_interests:
        responses['All Interests'].iloc[0].extend(add_interests)

    # Filter events based on user's interests
    events_df['Interest_Match'] = False
    for index, row in responses.iterrows():
        for tag in row['All Interests']:
            events_df.loc[events_df['Category'].apply(lambda x: tag in x), 'Interest_Match'] = True

    # Remove interests if specified
    if remove_interests:
        for interest in remove_interests:
            events_df.loc[events_df['Category'].apply(lambda x: interest in x), 'Interest_Match'] = False

    filtered_events_df = events_df[events_df['Interest_Match'] == True]

    # Flag events matching added interests
    if add_interests:
        filtered_events_df['Added_Interest_Match'] = filtered_events_df['Category'].apply(lambda x: any(interest in x for interest in add_interests))
        added_interest_events = filtered_events_df[filtered_events_df['Added_Interest_Match'] == True]
        other_interest_events = filtered_events_df[filtered_events_df['Added_Interest_Match'] == False]
    else:
        added_interest_events = pd.DataFrame()
        other_interest_events = filtered_events_df

    # Prioritize events matching added interests
    if add_interests:
        added_interest_events = added_interest_events.sort_values(by='Added_Interest_Match', ascending=False)

    filtered_interactions_df_1 = interacted_events[~interacted_events['Interaction Score'].isin([-1, 0])]
    merged_df = pd.merge(interacted_events, filtered_events_df[['Event ID', 'Primary Category', 'Sub Category']], on='Event ID', how='left', suffixes=('_int', '_evt'))
    merged_df_1 = pd.merge(filtered_interactions_df_1, filtered_events_df[['Event ID', 'Primary Category', 'Sub Category']], on='Event ID', how='left', suffixes=('_int', '_evt'))
    filtered_interactions_df_neg_zero = interacted_events[interacted_events['Interaction Score'].isin([-1, 0])]

    categories = merged_df.sort_values('Interaction Score', ascending=False)['Primary Category_evt'].unique()
    recommended_categories = set()

    for category in categories:
        category_events = filtered_events_df[filtered_events_df['Primary Category'] == category]
        user_subcategories = merged_df_1[merged_df_1['Primary Category_evt'] == category]['Sub Category_evt'].unique()
        for subcategory in user_subcategories:
            subcategory_events = category_events[category_events['Sub Category'] == subcategory]
            new_events = subcategory_events[~subcategory_events['Event ID'].isin(merged_df_1['Event ID']) &
                                            ~subcategory_events['Event ID'].isin(recommended_events['Event ID']) &
                                            ~subcategory_events['Event ID'].isin(filtered_interactions_df_neg_zero['Event ID'])]
            if not new_events.empty:
                recommended_categories.add(category)
                recommended_events = pd.concat([recommended_events, new_events], ignore_index=True)

    # Add unrecommended events
    unrecommended_categories = set(categories) - recommended_categories
    for ur_category in unrecommended_categories:
        category_events_1 = filtered_events_df[filtered_events_df['Primary Category'] == ur_category]
        user_subcategories_1 = merged_df[merged_df['Primary Category_evt'] == ur_category]['Sub Category_evt'].unique()
        for sc in user_subcategories_1:
            subcategory_events_1 = category_events_1[category_events_1['Sub Category'] != sc]
            ur_category_events = subcategory_events_1[
                (~subcategory_events_1['Event ID'].isin(merged_df['Event ID'])) &
                (~subcategory_events_1['Event ID'].isin(recommended_events['Event ID']))]
            recommended_events = pd.concat([recommended_events, ur_category_events], ignore_index=True)

    # Add additional events based on sorting
    sorted_events = merged_df.sort_values('Interaction Score', ascending=False)
    for index, event in sorted_events.iterrows():
        additional_events = filtered_events_df[
            (filtered_events_df['Primary Category'] == event['Primary Category_evt']) &
            (~filtered_events_df['Event ID'].isin(merged_df['Event ID'])) &
            (~filtered_events_df['Event ID'].isin(recommended_events['Event ID']))]
        recommended_events = pd.concat([recommended_events, additional_events], ignore_index=True)

    # Ensure unique events and retain priority
    recommended_events = recommended_events.drop_duplicates(subset='Event ID', keep='first')

    # Select top events based on specified ratio
    num_added_interest_events = int(20 * ratio_added_interests)
    num_other_interest_events = 20 - num_added_interest_events

    top_added_interest_events = added_interest_events.head(num_added_interest_events)
    top_other_interest_events = other_interest_events.head(num_other_interest_events)

    # Ensure we have enough events
    if len(top_added_interest_events) < num_added_interest_events:
        num_other_interest_events += num_added_interest_events - len(top_added_interest_events)
        top_added_interest_events = added_interest_events
    if len(top_other_interest_events) < num_other_interest_events:
        num_added_interest_events += num_other_interest_events - len(top_other_interest_events)
        top_other_interest_events = other_interest_events

    top_added_interest_events = added_interest_events.head(num_added_interest_events)
    top_other_interest_events = other_interest_events.head(num_other_interest_events)

    final_recommended_events = pd.concat([top_added_interest_events, top_other_interest_events])

    # Shuffle the final recommendations
    final_recommended_events = final_recommended_events.sample(frac=1, random_state=42).reset_index(drop=True)

    return final_recommended_events.head(20)

# Example usage
user = 'Pratik Bhande'
add_interests = ['none']
remove_interests = ['none']
ratio_added_interests = 0.6  # 60% of the top 20 events should be from added interests

recommended_events = recommend_events(user, events_df, responses, add_interests, remove_interests, ratio_added_interests)
recommended_events

C:\Users\pratik\AppData\Local\Temp\ipykernel_11372\2205694410.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  responses['All Interests'] = responses['All Interests'].str.split(', ')
C:\Users\pratik\AppData\Local\Temp\ipykernel_11372\2205694410.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_events_df['Added_Interest_Match'] = filtered_events_df['Category'].apply(lambda x: any(interest in x for interest in add_interests))


,Event ID,Event Names,Description,Primary Category,Secondary Category,Sub Category,Location,Prices,Date,Unnamed: 9,Time,Links,Category,Interest_Match,Added_Interest_Match
0,EVT0003,Ajay Atul Live,Let's enjoy the magic of Ajay-Atul’s music lik...,Music and Entertainment,NaN,Concert,Bavdhan,1500,2024-04-20 00:00:00,Saturday,19:00:00,https://in.bookmyshow.com/events/ajay-atul-liv...,[Music and Entertainment],True,False
1,EVT0031,Pharma and Lab Expo,NaN,Tech and Innovation,NaN,Exhibition,Kharadi,400,2024-07-14 00:00:00,Sunday,17:00:00,NaN,[Tech and Innovation],True,False
2,EVT0026,Musical Band and Orchestra,NaN,Music and Entertainment,NaN,Instrument,Paud Road,2000,2024-03-08 00:00:00,Friday,19:00:00,NaN,[Music and Entertainment],True,False
3,EVT0005,Poetry and Comedy,"GOTU LIT WEDNESDAY'S, A COMEDY AND POETRY OPEN...",Music and Entertainment,NaN,"Poetry,Comedy",Viman Nagar,400,2024-03-24 00:00:00,Sunday,17:00:00,NaN,[Music and Entertainment],True,False
4,EVT0018,Global Tech Engineering Expo,GMCATALYSIS2024 is an annual meeting organized...,Tech and Innovation,NaN,Exhibition,Ravet,0,2024-03-30 00:00:00,Saturday,16:00:00,NaN,[Tech and Innovation],True,False
5,EVT0015,Katraj to Sinhagad Trek,Katraj to Sinhgad trek is famous as K2S trek. ...,Adventure,Travel,Hiking and Trekking,Katraj,1500,2024-03-24 00:00:00,Sunday,17:30:00,https://www.treksandtrails.org/tours/katraj-to...,"[Adventure, Travel]",True,False
6,EVT0022,Fun and Fair,"Don’t miss new attractions, the thrills of the...",Music and Entertainment,Food and Drinks,Exhibition,Erandwane,0,23/3/24 to 31/3/24,23/3/24 to 31/3/24,15:00:00,NaN,"[Music and Entertainment, Food and Drinks]",True,False
7,EVT0010,Career Guidance in Sports Management & Sports ...,IISM would like to invite you for a seminar on...,Books and Education,Sports,Academic,Balewadi,0,2024-06-04 00:00:00,Tuesday,12:00:00,https://allevents.in/pune/seminar-on-career-gu...,"[Books and Education, Sports]",True,False
8,EVT0033,Panshet Water Sports,NaN,Adventure,NaN,Adventure Sports,Kharadi,599,2024-10-04 00:00:00,Friday,07:00:00,NaN,[Adventure],True,False
9,EVT0029,Global Tech Medical Expo,NaN,Tech and Innovation,NaN,Exhibition,Viman Nagar,300,2024-11-29 00:00:00,Friday,07:00:00,NaN,[Tech and Innovation],True,False


### Code Flow and Explanation:

1. **Import Libraries**:
   - `pandas`, `cosine_similarity`, `numpy`, `random`.

2. **Load Data**:
   - Load interaction data (`interactions_df`), event details (`events_df`), and user responses (`responses`).

3. **Create User-Item Matrix**:
   - A matrix where rows = users, columns = events, values = interaction scores. Missing values are filled with 0.

4. **User Similarity**:
   - Compute cosine similarity between users based on their event interactions.

5. **Recommendation Function `recommend_events()`**:
   - **Collaborative Filtering**: Identify similar users and recommend events they've interacted with that the current user hasn't.
   - **Content-Based Filtering**: Match events to user interests, allowing for adding/removing specific interests.
   - **Prioritization**: Balance between collaborative recommendations and content-based filtering (user interests).
   - **Final Recommendations**: Ensure a mix of events based on added interests and others, sort, and return the top 20 events.


